## Huấn luyện mô hình CBoW

Notebook này là một phần của [Chương trình học AI cho người mới bắt đầu](http://aka.ms/ai-beginners)

Trong ví dụ này, chúng ta sẽ tìm hiểu cách huấn luyện mô hình ngôn ngữ CBoW để tạo không gian nhúng Word2Vec của riêng mình. Chúng ta sẽ sử dụng tập dữ liệu AG News làm nguồn văn bản.


In [ ]:
import torch
import torchtext
import os
import collections
import builtins
import random
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Đầu tiên, hãy tải bộ dữ liệu của chúng ta và định nghĩa bộ mã hóa và từ vựng. Chúng ta sẽ đặt `vocab_size` là 5000 để giới hạn tính toán một chút.


In [ ]:
def load_dataset(ngrams = 1, min_freq = 1, vocab_size = 5000 , lines_cnt = 500):
    tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
    print("Loading dataset...")
    test_dataset, train_dataset  = torchtext.datasets.AG_NEWS(root='./data')
    train_dataset = list(train_dataset)
    test_dataset = list(test_dataset)
    classes = ['World', 'Sports', 'Business', 'Sci/Tech']
    print('Building vocab...')
    counter = collections.Counter()
    for i, (_, line) in enumerate(train_dataset):
        counter.update(torchtext.data.utils.ngrams_iterator(tokenizer(line),ngrams=ngrams))
        if i == lines_cnt:
            break
    vocab = torchtext.vocab.Vocab(collections.Counter(dict(counter.most_common(vocab_size))), min_freq=min_freq)
    return train_dataset, test_dataset, classes, vocab, tokenizer

In [ ]:
train_dataset, test_dataset, _, vocab, tokenizer = load_dataset()

Loading dataset...
Building vocab...


In [ ]:
def encode(x, vocabulary, tokenizer = tokenizer):
    return [vocabulary[s] for s in tokenizer(x)]

## Mô hình CBoW

CBoW học cách dự đoán một từ dựa trên $2N$ từ lân cận. Ví dụ, khi $N=1$, chúng ta sẽ có các cặp sau từ câu *I like to train networks*: (like,I), (I, like), (to, like), (like,to), (train,to), (to, train), (networks, train), (train,networks). Ở đây, từ đầu tiên là từ lân cận được sử dụng làm đầu vào, và từ thứ hai là từ mà chúng ta đang dự đoán.

Để xây dựng một mạng dự đoán từ tiếp theo, chúng ta cần cung cấp từ lân cận làm đầu vào và nhận số thứ tự của từ làm đầu ra. Kiến trúc của mạng CBoW như sau:

* Từ đầu vào được đưa qua lớp nhúng (embedding layer). Chính lớp nhúng này sẽ là nhúng Word2Vec của chúng ta, do đó chúng ta sẽ định nghĩa nó riêng biệt dưới dạng biến `embedder`. Trong ví dụ này, chúng ta sẽ sử dụng kích thước nhúng = 30, mặc dù bạn có thể muốn thử nghiệm với các kích thước lớn hơn (Word2Vec thực tế có kích thước 300).
* Vector nhúng sau đó sẽ được đưa qua một lớp tuyến tính để dự đoán từ đầu ra. Do đó, lớp này sẽ có số lượng neuron bằng `vocab_size`.

Đối với đầu ra, nếu chúng ta sử dụng `CrossEntropyLoss` làm hàm mất mát, chúng ta cũng chỉ cần cung cấp số thứ tự của từ làm kết quả mong đợi, mà không cần mã hóa one-hot.


In [ ]:
vocab_size = len(vocab)

embedder = torch.nn.Embedding(num_embeddings = vocab_size, embedding_dim = 30)
model = torch.nn.Sequential(
    embedder,
    torch.nn.Linear(in_features = 30, out_features = vocab_size),
)

print(model)

Sequential(
  (0): Embedding(5002, 30)
  (1): Linear(in_features=30, out_features=5002, bias=True)
)


## Chuẩn bị dữ liệu huấn luyện

Bây giờ, hãy lập trình hàm chính để tính các cặp từ CBoW từ văn bản. Hàm này sẽ cho phép chúng ta xác định kích thước cửa sổ và trả về một tập hợp các cặp - từ đầu vào và từ đầu ra. Lưu ý rằng hàm này có thể được sử dụng cho các từ, cũng như cho các vector/tensor - điều này sẽ cho phép chúng ta mã hóa văn bản trước khi truyền nó vào hàm `to_cbow`.


In [ ]:
def to_cbow(sent,window_size=2):
    res = []
    for i,x in enumerate(sent):
        for j in range(max(0,i-window_size),min(i+window_size+1,len(sent))):
            if i!=j:
                res.append([sent[j],x])
    return res

print(to_cbow(['I','like','to','train','networks']))
print(to_cbow(encode('I like to train networks', vocab)))

[['like', 'I'], ['to', 'I'], ['I', 'like'], ['to', 'like'], ['train', 'like'], ['I', 'to'], ['like', 'to'], ['train', 'to'], ['networks', 'to'], ['like', 'train'], ['to', 'train'], ['networks', 'train'], ['to', 'networks'], ['train', 'networks']]
[[232, 172], [5, 172], [172, 232], [5, 232], [0, 232], [172, 5], [232, 5], [0, 5], [1202, 5], [232, 0], [5, 0], [1202, 0], [5, 1202], [0, 1202]]


Hãy chuẩn bị tập dữ liệu huấn luyện. Chúng ta sẽ đi qua tất cả các tin tức, gọi `to_cbow` để lấy danh sách các cặp từ, và thêm những cặp từ đó vào `X` và `Y`. Để tiết kiệm thời gian, chúng ta sẽ chỉ xem xét 10k tin tức đầu tiên - bạn có thể dễ dàng loại bỏ giới hạn này nếu có nhiều thời gian chờ đợi hơn và muốn có các embedding tốt hơn :)


In [ ]:
X = []
Y = []
for i, x in zip(range(10000), train_dataset):
    for w1, w2 in to_cbow(encode(x[1], vocab), window_size = 5):
        X.append(w1)
        Y.append(w2)

X = torch.tensor(X)
Y = torch.tensor(Y)

Chúng tôi cũng sẽ chuyển đổi dữ liệu đó thành một tập dữ liệu và tạo dataloader:


In [ ]:
class SimpleIterableDataset(torch.utils.data.IterableDataset):
    def __init__(self, X, Y):
        super(SimpleIterableDataset).__init__()
        self.data = []
        for i in range(len(X)):
            self.data.append( (Y[i], X[i]) )
        random.shuffle(self.data)

    def __iter__(self):
        return iter(self.data)

Chúng tôi cũng sẽ chuyển đổi dữ liệu đó thành một tập dữ liệu và tạo dataloader:


In [ ]:
ds = SimpleIterableDataset(X, Y)
dl = torch.utils.data.DataLoader(ds, batch_size = 256)

Bây giờ hãy bắt đầu quá trình huấn luyện thực tế. Chúng ta sẽ sử dụng bộ tối ưu hóa `SGD` với tốc độ học khá cao. Bạn cũng có thể thử nghiệm với các bộ tối ưu hóa khác, chẳng hạn như `Adam`. Chúng ta sẽ huấn luyện trong 10 epoch để bắt đầu - và bạn có thể chạy lại ô này nếu muốn giảm tổn thất hơn nữa.


In [ ]:
def train_epoch(net, dataloader, lr = 0.01, optimizer = None, loss_fn = torch.nn.CrossEntropyLoss(), epochs = None, report_freq = 1):
    optimizer = optimizer or torch.optim.Adam(net.parameters(), lr = lr)
    loss_fn = loss_fn.to(device)
    net.train()

    for i in range(epochs):
        total_loss, j = 0, 0, 
        for labels, features in dataloader:
            optimizer.zero_grad()
            features, labels = features.to(device), labels.to(device)
            out = net(features)
            loss = loss_fn(out, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss
            j += 1
        if i % report_freq == 0:
            print(f"Epoch: {i+1}: loss={total_loss.item()/j}")

    return total_loss.item()/j

In [ ]:
train_epoch(net = model, dataloader = dl, optimizer = torch.optim.SGD(model.parameters(), lr = 0.1), loss_fn = torch.nn.CrossEntropyLoss(), epochs = 10)

Epoch: 1: loss=5.664632366860172
Epoch: 2: loss=5.632101973960962
Epoch: 3: loss=5.610399051405015
Epoch: 4: loss=5.594621561080262
Epoch: 5: loss=5.582538017415446
Epoch: 6: loss=5.572900234519603
Epoch: 7: loss=5.564951676341915
Epoch: 8: loss=5.558288112064614
Epoch: 9: loss=5.552576955031129
Epoch: 10: loss=5.547634165194347


5.547634165194347

## Thử nghiệm Word2Vec

Để sử dụng Word2Vec, hãy trích xuất các vector tương ứng với tất cả các từ trong từ vựng của chúng ta:


In [ ]:
vectors = torch.stack([embedder(torch.tensor(vocab[s])) for s in vocab.itos], 0)

Hãy xem, ví dụ, cách từ **Paris** được mã hóa thành một vector:


In [ ]:
paris_vec = embedder(torch.tensor(vocab['paris']))
print(paris_vec)

tensor([-0.0915,  2.1224, -0.0281, -0.6819,  1.1219,  0.6458, -1.3704, -1.3314,
        -1.1437,  0.4496,  0.2301, -0.3515, -0.8485,  1.0481,  0.4386, -0.8949,
         0.5644,  1.0939, -2.5096,  3.2949, -0.2601, -0.8640,  0.1421, -0.0804,
        -0.5083, -1.0560,  0.9753, -0.5949, -1.6046,  0.5774],
       grad_fn=<EmbeddingBackward>)


Thật thú vị khi sử dụng Word2Vec để tìm kiếm các từ đồng nghĩa. Hàm sau đây sẽ trả về `n` từ gần nhất với một từ đầu vào cho trước. Để tìm chúng, chúng ta tính chuẩn của $|w_i - v|$, trong đó $v$ là vector tương ứng với từ đầu vào của chúng ta, và $w_i$ là mã hóa của từ thứ $i$ trong từ vựng. Sau đó, chúng ta sắp xếp mảng và trả về các chỉ số tương ứng bằng cách sử dụng `argsort`, và lấy `n` phần tử đầu tiên của danh sách, những phần tử này mã hóa vị trí của các từ gần nhất trong từ vựng.


In [ ]:
def close_words(x, n = 5):
  vec = embedder(torch.tensor(vocab[x]))
  top5 = np.linalg.norm(vectors.detach().numpy() - vec.detach().numpy(), axis = 1).argsort()[:n]
  return [ vocab.itos[x] for x in top5 ]

close_words('microsoft')

['microsoft', 'quoted', 'lp', 'rate', 'top']

In [ ]:
close_words('basketball')

['basketball', 'lot', 'sinai', 'states', 'healthdaynews']

In [ ]:
close_words('funds')

['funds', 'travel', 'sydney', 'japan', 'business']

## Điểm chính

Bằng cách sử dụng các kỹ thuật thông minh như CBoW, chúng ta có thể huấn luyện mô hình Word2Vec. Bạn cũng có thể thử huấn luyện mô hình skip-gram, được thiết kế để dự đoán từ lân cận dựa trên từ trung tâm, và xem hiệu suất của nó như thế nào.



---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn thông tin chính thức. Đối với các thông tin quan trọng, khuyến nghị sử dụng dịch vụ dịch thuật chuyên nghiệp bởi con người. Chúng tôi không chịu trách nhiệm cho bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
